In [458]:
import numpy as np
import matplotlib.pyplot as plt
import fluids as fl
import sympy as sp
from pint import _DEFAULT_REGISTRY as u

In [459]:
class propFeed():
    def __init__(self):
        self.componentName = None
        self.componentNumber = None #For sorting later

In [460]:
class liquids(propFeed):
    def __init__(self, propellantName, liquidDensity, liquidViscosity, liquidMassFlow):
        self.propellantName = propellantName
        self.liquidDensity = liquidDensity
        self.liquidViscosity = liquidViscosity
        self.liquidMassFlow = liquidMassFlow

    #Establishing sorting criteria
    def __lt__(self, other):
        return self.componentNumber < other.componentNumber

In [461]:
class liquidValves(liquids):
    def __init__(self, ID, Cv, componentNumber, componentName, **args):
        self.Cv = Cv
        self.ID = ID
        self.area = np.pi*((ID/2)**2)
        
        #Runs the superclass constructor with the additional arguments (which are meant to be the unpacked attributes of a liquid profile)
        super().__init__(**args)
        self.componentNumber = componentNumber
        self.componentName = componentName

        self.velocity = self.liquidMassFlow / (self.liquidDensity * self.area)
        self.K = fl.units.Cv_to_K(Cv, ID)
        self.pressureDrop = (fl.units.dP_from_K(K = self.K, rho = self.liquidDensity, V = self.velocity)).to("psi")

    #def KFactorFunc(self):
        #K, Cv, ID = sp.symbols("K, Cv, ID", real=True)
        #eqn = sp.Eq(K, (1.6 * 10**9)*((ID**4)/(Cv/1.56)**2))
        #solutionList = sp.solve(eqn, K)
        #solution = solutionList[0].subs([(Cv, self.Cv), (ID, self.Cv)])
        #print(solution)

In [462]:
class liquidBends(liquids):
    def __init__(self, ID, bendAngle, bendRadius, roughness, componentNumber, componentName, **args):
        self.ID = ID
        self.angle = bendAngle
        self.bendRadius = bendRadius
        self.roughness = roughness
        self.area = np.pi*((ID/2)**2)
        
        #Runs the superclass constructor with the additional arguments (which are meant to be the unpacked attributes of a liquid profile)
        super().__init__(**args)
        self.componentNumber = componentNumber
        self.componentName = componentName

        self.velocity = self.liquidMassFlow / (self.liquidDensity * self.area)
        self.reynolds = fl.units.Reynolds(V = self.velocity, D = self.ID, rho = self.liquidDensity, mu = self.liquidViscosity)
        self.frictionFactor = fl.units.friction_factor(Re = self.reynolds, eD = (self.roughness/self.ID), Darcy = True)
        self.K = fl.units.bend_rounded(Di = self.ID, angle = self.angle, rc = self.bendRadius, fd = self.frictionFactor, Re = self.reynolds, roughness = self.roughness, method = 'Rennels')
        self.pressureDrop = (fl.units.dP_from_K(K = self.K, rho = self.liquidDensity, V = self.velocity)).to("psi")


In [463]:
#Create a liquid object "profile"
LOX = liquids(propellantName = "LOX", liquidDensity = 1100*(u.kilogram/u.meter**3), liquidViscosity = 274.41*(u.micropascal*u.second), liquidMassFlow = 2*(u.kilogram/u.second))
Kerosene = liquids(propellantName = "Kerosene", liquidDensity = 0.81*(u.gram/u.centimeter**3), liquidViscosity = 0.00164*(u.pascal*u.second), liquidMassFlow = 0.91*(u.kilogram/u.second))

#Create objects of various classes
#Notice that the first argument is always an unpacked dict of the component's corresponding liquid profile, followed by arguments unique to that class of components
LOXValve1 = liquidValves(**LOX.__dict__, ID = 0.25*(u.inch), Cv = 6*(u.gallon/u.minute), componentNumber = 1, componentName = "LOXValve1")
LOXBend1 = liquidBends(**LOX.__dict__, ID = 0.25*(u.inch), bendAngle = 45*(u.degree), bendRadius = 1*(u.inch), roughness = 1*(u.micron), componentNumber = 2, componentName = "LOXBend1")

print("LOXValve1:")
print(f"ID: {LOXValve1.ID}")
print(f"Area: {LOXValve1.area}")
print(f"Velocity: {LOXValve1.velocity}")
print(f"KFactor: {LOXValve1.K}")
print(f"Density: {LOXValve1.liquidDensity}")
print(f"PressureDrop: {LOXValve1.pressureDrop}\n")

print("LOXBend1:")
print(f"ID: {LOXBend1.ID}")
print(f"Area: {LOXBend1.area}")
print(f"Velocity: {LOXBend1.velocity}")
print(f"KFactor: {LOXBend1.K}")
print(f"Density: {LOXBend1.liquidDensity}")
print(f"PressureDrop: {LOXBend1.pressureDrop}")


LOXValve1:
ID: 0.25 inch
Area: 0.04908738521234052 inch ** 2
Velocity: 0.037039695846841095 meter ** 3 / inch ** 2 / second
KFactor: 0.09658342579659235 dimensionless
Density: 1100 kilogram / meter ** 3
PressureDrop: 25.394886105842872 pound_force_per_square_inch

LOXBend1:
ID: 0.25 inch
Area: 0.04908738521234052 inch ** 2
Velocity: 0.037039695846841095 meter ** 3 / inch ** 2 / second
KFactor: 0.11758773100581164 dimensionless
Density: 1100 kilogram / meter ** 3
PressureDrop: 30.917592865529024 pound_force_per_square_inch


In [464]:
components = [LOXBend1, LOXValve1]
sortedComponents = sorted(components) #sorts components

#Create initial pressure and array to keep track of pressure
initialPressure = 800*(u.psi)
pressureTracker = []
pressureTracker.append(initialPressure)

#Iterates through every component
for component in sortedComponents:
    print(f"Name: {component.componentName}, Component Number: {component.componentNumber}")
    print(f"Inlet Pressure: {initialPressure}")

    #Update Pressure
    initialPressure = initialPressure - component.pressureDrop
    pressureTracker.append(initialPressure)

    print(f"Outlet Pressure: {initialPressure}")
    print(f"Pressure Drop: {component.pressureDrop}\n")

print(pressureTracker)

Name: LOXValve1, Component Number: 1
Inlet Pressure: 800 pound_force_per_square_inch
Outlet Pressure: 774.6051138941572 pound_force_per_square_inch
Pressure Drop: 25.394886105842872 pound_force_per_square_inch

Name: LOXBend1, Component Number: 2
Inlet Pressure: 774.6051138941572 pound_force_per_square_inch
Outlet Pressure: 743.6875210286281 pound_force_per_square_inch
Pressure Drop: 30.917592865529024 pound_force_per_square_inch

[<Quantity(800, 'pound_force_per_square_inch')>, <Quantity(774.605114, 'pound_force_per_square_inch')>, <Quantity(743.687521, 'pound_force_per_square_inch')>]
